# Dia 6 - GroupBy avanzado

## Agrupaciones con múltiples columnas

## 1) Introducción 

#### Tenés un dataset con miles de filas. Las preguntas del jefe ya no son “¿cuánto vendimos?”, sino:

- #### ¿Cuánto vendimos por sucursal y por categoría?

- #### ¿Cuál es el promedio del préstamo por región y por tipo de cliente?

- #### ¿Cuántas operaciones hay por mes y por estado (aprobado/rechazado)?

#### Leer fila por fila es imposible. Necesitamos resumir por grupos y, muchas veces, por más de una columna a la vez.
---

## 2) La herramienta que vamos a usar

- #### `groupby([...])` de pandas para agrupar por una o varias columnas.

### `¿Qué es groupby?`

- #### groupby no es un método que viene adentro de la librería pandas.

- #### O sea: cuando instalás pandas, ya viene incluido

### ¿Qué hace?

- #### Sirve para agrupar filas de un DataFrame según el valor de una o más columnas.

- #### Después de agrupar, podés hacer operaciones sobre cada grupo: sumar, contar, sacar promedio, etc.

#### Luego, aplicamos agregaciones que ya conocemos: `sum()`, `mean()`, `count()` (y más adelante agg combinado).

#### Pieza clave de hoy: `df.groupby(["colA", "colB"])` → crea grupos por cada combinación única de `colA` y `colB`.
---

## 3) Concepto

- #### **Agrupar** significa: juntar todas las filas que tienen el mismo valor en una columna.
#### Ejemplo: si tengo una columna “Sucursal” y varias filas dicen “Norte”, todas esas filas se juntan en un solo grupo llamado **Norte**.

- ####  **Con varias columnas**: el grupo no tiene un solo nombre, sino dos (o más).
#### Es como poner etiquetas dobles:

- #### (“Norte”, “Electrónica”) → todas las filas que sean sucursal Norte y categoría Electrónica.
- #### (“Sur”, “Hogar”) → todas las filas que sean sucursal Sur y categoría Hogar.

- #### Eso se llama **tupla**: un paquetito con varios valores juntos, como un nombre compuesto.

- #### **El resultado de groupby** te devuelve algo como una tabla con “índice doble”:

- #### La primera llave `(colA)` → Sucursal.
- #### La segunda llave `(colB)` → Categoría.
#### Así podés entrar primero por Sucursal y después por Categoría.

- #### * Si querés que esa tabla quede **normalita, con un solo índice**, usás `.reset_index()`.
#### Eso “aplana” la tabla y vuelve a poner las columnas visibles como antes.

---

### En resumen:

- #### Agrupar = juntar filas que coinciden.
- #### Con varias columnas = el grupo se nombra con varios valores a la vez.
- #### El resultado queda con un “doble índice”.
- #### Si molesta, se aplana con `.reset_index()`.

---


## 4) Ejemplos en código 
### Ejemplo A — Ventas por Sucursal y Categoría

In [ ]:
import pandas as pd

data = {
    'Sucursal': ['Norte', 'Norte', 'Sur', 'Sur', 'Este', 'Este', 'Oeste', 'Oeste'],
    'Categoria': ['Electronica', 'Hogar', 'Electronica', 'Hogar', 'Hogar', 'Ropa', 'Electronica', 'Ropa'],
    'Monto': [3500,1800,1200,2500,2100,1300,2300,900]
}

df = pd.DataFrame(data)
print(df)

print('')

# total vendido por sucursal y categoria
resumen = df.groupby(['Sucursal','Categoria'])['Monto'].sum()
print(resumen)

print('')

# para resetear la tabla porque queremos volver a utilizar y trabajar con normalidad
print(resumen.reset_index())

  Sucursal    Categoria  Monto
0    Norte  Electronica   3500
1    Norte        Hogar   1800
2      Sur  Electronica   1200
3      Sur        Hogar   2500
4     Este        Hogar   2100
5     Este         Ropa   1300
6    Oeste  Electronica   2300
7    Oeste         Ropa    900

Sucursal  Categoria  
Este      Hogar          2100
          Ropa           1300
Norte     Electronica    3500
          Hogar          1800
Oeste     Electronica    2300
          Ropa            900
Sur       Electronica    1200
          Hogar          2500
Name: Monto, dtype: int64

  Sucursal    Categoria  Monto
0     Este        Hogar   2100
1     Este         Ropa   1300
2    Norte  Electronica   3500
3    Norte        Hogar   1800
4    Oeste  Electronica   2300
5    Oeste         Ropa    900
6      Sur  Electronica   1200
7      Sur        Hogar   2500


### Ejemplo B — Préstamos: Promedio y Cantidad por Región y Tipo de Cliente

In [7]:
data = {
    "Region": ["Norte","Norte","Sur","Sur","Sur","Este","Oeste","Oeste"],
    "Tipo":   ["Minorista","Corporativo","Minorista","Corporativo","Minorista","Corporativo","Minorista","Corporativo"],
    "Monto":  [1000,2000,1500,3000,1200,2500,900,1800]
}
df = pd.DataFrame(data)

# Promedio y cantidad por combinación (Region, Tipo)
prom = df.groupby(["Region","Tipo"])["Monto"].mean()
cnt  = df.groupby(["Region","Tipo"])["Monto"].count()

print("Promedio por grupo:")
print(prom.reset_index(name="Promedio_Monto"))

print("\nCantidad por grupo:")
print(cnt.reset_index(name="Cantidad"))


Promedio por grupo:
  Region         Tipo  Promedio_Monto
0   Este  Corporativo          2500.0
1  Norte  Corporativo          2000.0
2  Norte    Minorista          1000.0
3  Oeste  Corporativo          1800.0
4  Oeste    Minorista           900.0
5    Sur  Corporativo          3000.0
6    Sur    Minorista          1350.0

Cantidad por grupo:
  Region         Tipo  Cantidad
0   Este  Corporativo         1
1  Norte  Corporativo         1
2  Norte    Minorista         1
3  Oeste  Corporativo         1
4  Oeste    Minorista         1
5    Sur  Corporativo         1
6    Sur    Minorista         2


## 5) Errores típicos (y cómo evitarlos)

### Olvidar seleccionar una columna numérica antes de sumar/promediar:
- #### `df.groupby(["A","B"]).sum()` suma todas las numéricas; si querés solo una, hacé `["col"]`.

- #### Asustarse por el MultiIndex: es normal. Si necesitás un índice común, usá `.reset_index()`.

- #### Nulos: `count()` no cuenta NaN. Si hay faltantes, tus totales/cantidades pueden sorprender. Revisar con `df.isna().sum()`.
---

## 6) Lectura e interpretación (cómo “leer” el resultado)

- #### Cada fila del resultado representa una combinación única de las columnas por las que agrupaste.

- #### La columna agregada (ej. Monto) muestra el resumen para esa combinación (suma, promedio, conteo).

- #### Si ves un MultiIndex, pensalo como “título y subtítulo”: primero Sucursal, dentro Categoria.
---

## Ejercicio único (GroupBy multi-columna, corto)

### **Dataset:** `Fecha`, `Sucursal` (Norte/Sur/Este/Oeste), `Categoria` (Electrónica/Hogar/Ropa), `Precio`, `Cantidad`.

#### 1. Crear `Ingreso = Precio * Cantidad`.
#### 2. `groupby(["Sucursal","Categoria"])` y calcular:

- #### **Ingreso\_total** = `sum` de `Ingreso`
- #### **Precio\_prom** = `mean` de `Precio`
- #### **Ops** = `count` (cualquier columna)
#### Luego `reset_index()`.
#### 3. **Sin ordenar**, responder:

- #### ¿Qué (Sucursal, Categoria) tiene **Ingreso\_total** máximo? (usá `.max()` y filtrá esa fila)
- #### ¿Dónde el **Precio\_prom** es máximo? (igual: `.max()` + filtro)
#### 4. Repetí 1–3 **solo** con filas `Cantidad >= 3` **y** `Categoria != "Ropa"`.

- #### Decí en **1 línea** qué cambió y por qué.
---


### 1) Armar dataset y columna derivada

In [10]:
import pandas as pd

data = {
    "Fecha": ["2025-01-01","2025-01-02","2025-01-03","2025-01-04","2025-01-05","2025-01-06"],
    "Sucursal": ["Norte","Norte","Sur","Sur","Este","Oeste"],
    "Categoria": ["Electrónica","Hogar","Electrónica","Hogar","Ropa","Electrónica"],
    "Precio": [3500,1800,1200,2500,1300,2300],
    "Cantidad": [2,3,5,1,4,2]
}
df = pd.DataFrame(data)

# Columna derivada
df["Ingreso"] = df["Precio"] * df["Cantidad"]


## 2) Agrupación multi-columna

In [11]:
resumen = df.groupby(["Sucursal","Categoria"]).agg(
    Ingreso_total=("Ingreso","sum"),
    Precio_prom=("Precio","mean"),
    Ops=("Fecha","count")
).reset_index()

print(resumen)


  Sucursal    Categoria  Ingreso_total  Precio_prom  Ops
0     Este         Ropa           5200       1300.0    1
1    Norte  Electrónica           7000       3500.0    1
2    Norte        Hogar           5400       1800.0    1
3    Oeste  Electrónica           4600       2300.0    1
4      Sur  Electrónica           6000       1200.0    1
5      Sur        Hogar           2500       2500.0    1


## 3) Buscar máximos

In [12]:
# Máximo ingreso total
max_ing = resumen["Ingreso_total"].max()
print(resumen[resumen["Ingreso_total"] == max_ing])

# Máximo precio promedio
max_prec = resumen["Precio_prom"].max()
print(resumen[resumen["Precio_prom"] == max_prec])


  Sucursal    Categoria  Ingreso_total  Precio_prom  Ops
1    Norte  Electrónica           7000       3500.0    1
  Sucursal    Categoria  Ingreso_total  Precio_prom  Ops
1    Norte  Electrónica           7000       3500.0    1


### Con estos datos:

- #### Ingreso_total máximo → Sur, Electrónica (Ingreso_total = 6000).

- #### Precio_prom máximo → Norte, Electrónica (Precio_prom = 3500).
---

## 4) Repetir con filtro (Cantidad >= 3 y Categoria != "Ropa")

In [13]:
filtro = df[(df["Cantidad"] >= 3) & (df["Categoria"] != "Ropa")]
resumen_f = filtro.groupby(["Sucursal","Categoria"]).agg(
    Ingreso_total=("Ingreso","sum"),
    Precio_prom=("Precio","mean"),
    Ops=("Fecha","count")
).reset_index()

print(resumen_f)


  Sucursal    Categoria  Ingreso_total  Precio_prom  Ops
0    Norte        Hogar           5400       1800.0    1
1      Sur  Electrónica           6000       1200.0    1


## Ahora:

- #### Quedan solo Norte-Hogar y Sur-Electrónica.

- #### Máximo ingreso sigue siendo Sur, Electrónica pero con menos datos.
---

## 5) Conclusión en 1 línea

#### Al aplicar el filtro se reducen las combinaciones válidas → cambian los totales y desaparecen las categorías que no cumplen condiciones.
---
---

## Ejercicio 2 — Préstamos por Región y Tramo de Monto
#### Dataset con: Cliente, Region (N/E/S/O), Monto, Plazo.

#### 1. Crear columna Tramo con 3 rangos de monto (ej.: “Bajo” ≤1500, “Medio” 1501–2500, “Alto” >2500) usando condiciones simples.

#### 2. Agrupar por Region y Tramo y calcular:

- #### Suma de Monto.

- #### Promedio de Plazo.

- #### Cantidad de operaciones.

#### 3. Con una sola frase por ítem, interpretar:

- #### ¿Qué región concentra más monto en el tramo “Alto”?

- #### ¿En qué tramo los plazos promedio resultan mayores y por qué podría pasar?

#### (Pistas: crear `Tramo` con `np.where`/condiciones encadenadas o equivalentes sencillos; agrupar con `groupby(["Region","Tramo"])`; `sum()`, `mean()`, `count()`; `reset_index()`.)

## Ejercicio 2 - 1) Dataset + columna Tramo

In [14]:
import pandas as pd
import numpy as np

data = {
    "Cliente": ["Ana","Luis","Sofía","Marta","Juan","Carla","Pedro","Laura"],
    "Region":  ["N","N","S","S","E","E","O","O"],
    "Monto":   [1000, 2800, 1500, 3200, 2200, 1300, 2700, 1800],
    "Plazo":   [12, 24, 18, 36, 24, 12, 30, 18]
}
df = pd.DataFrame(data)

# Crear columna Tramo
df["Tramo"] = np.where(df["Monto"] <= 1500, "Bajo",
                np.where(df["Monto"] <= 2500, "Medio", "Alto"))

print(df[["Cliente","Region","Monto","Plazo","Tramo"]])


  Cliente Region  Monto  Plazo  Tramo
0     Ana      N   1000     12   Bajo
1    Luis      N   2800     24   Alto
2   Sofía      S   1500     18   Bajo
3   Marta      S   3200     36   Alto
4    Juan      E   2200     24  Medio
5   Carla      E   1300     12   Bajo
6   Pedro      O   2700     30   Alto
7   Laura      O   1800     18  Medio


## 2) Agrupar por Región y Tramo

In [15]:
resumen = df.groupby(["Region","Tramo"]).agg(
    Monto_total=("Monto","sum"),
    Plazo_prom=("Plazo","mean"),
    Operaciones=("Cliente","count")
).reset_index()

print(resumen)


  Region  Tramo  Monto_total  Plazo_prom  Operaciones
0      E   Bajo         1300        12.0            1
1      E  Medio         2200        24.0            1
2      N   Alto         2800        24.0            1
3      N   Bajo         1000        12.0            1
4      O   Alto         2700        30.0            1
5      O  Medio         1800        18.0            1
6      S   Alto         3200        36.0            1
7      S   Bajo         1500        18.0            1


## 3) Interpretaciones

#### 1. ¿Qué región concentra más monto en el tramo “Alto”?
- #### La región Sur concentra más monto en el tramo Alto porque tiene el préstamo más grande (3200).

#### 2. ¿En qué tramo los plazos promedio resultan mayores y por qué podría pasar?
- #### En el tramo Alto, porque los préstamos grandes suelen darse a plazos más largos para que sean más pagables.
---

## 8) Cierre

#### Agrupar por múltiples columnas es el paso natural cuando las preguntas del negocio son bidimensionales (o más): por región y categoría, 
#### por mes y estado, por sucursal y canal.
#### Hoy viste cómo construir esos grupos, agregar métricas y leer el resultado. En el próximo bloque, vamos a combinar agregaciones en una sola instrucción con agg, 
#### para producir varias métricas a la vez de forma limpia y eficiente.
---

# Tema 2: Agregaciones combinadas (agg)

## 1) Introducción 

#### Cuando trabajamos con datos reales, no alcanza con mirar un solo número. En un análisis serio casi siempre necesitamos varias métricas al mismo tiempo: totales, promedios, conteos, mínimos o máximos. Cada una de esas medidas nos da un ángulo distinto de la información y juntas nos permiten ver el panorama completo.

### Hasta ahora, lo que hicimos fue aplicar estas métricas por separado. Por ejemplo:

- #### usamos `sum()` para calcular los totales,

- #### después aplicamos `mean()` para obtener los promedios,

- #### y más tarde `count()` para conocer la cantidad de operaciones.

#### Este enfoque funciona, pero tiene un problema: es repetitivo. Para cada cálculo hay que volver a agrupar los datos y eso se vuelve lento, confuso y difícil de leer cuando el análisis crece.

#### Imaginemos una situación típica: el jefe nos pide un informe que muestre el total de préstamos, el promedio y la cantidad de operaciones por sucursal y tipo de cliente.
#### Si intentamos resolverlo con lo que veníamos haciendo, tendríamos que escribir tres pasos distintos, repetir el groupby varias veces y luego combinar manualmente los resultados.

#### Claramente eso no es eficiente. Lo ideal sería poder hacer todos esos cálculos juntos, en una sola pasada, y obtener una tabla compacta con todas las métricas al lado.
---

## 2) La herramienta (agg)

#### La función `.agg()` es de **pandas**, no de Python puro. Se usa después de un `groupby()` y permite aplicar varias operaciones de resumen al mismo tiempo 
#### (como suma, promedio y conteo) en una sola pasada. Esto hace el código más claro y rápido, porque no tenés que repetir el agrupamiento varias veces.

#### `Una analogía simple:` es como un profesor que mira las notas de un grupo y, de una sola vez, anota el promedio, la nota más alta y la más baja, sin tener que revisar la libreta tres veces.
---


### Sintaxis básica:

## `df.groupby("col")["otra_col"].agg(["sum","mean","count"])`


### También se puede usar un diccionario para personalizar nombres:

### `df.groupby("col").agg(`
###     `Total_Monto=("Monto","sum"),`
###     `Promedio_Monto=("Monto","mean"),`
###     `Operaciones=("Cliente","count")`
### `)`
---


## 3) Concepto en palabras simples

- #### agg viene de aggregate, que significa juntar o resumir datos.

- #### Con agg podés pedir varias cuentas al mismo tiempo (suma, promedio, cantidad, etc.) dentro de un mismo `groupby()`.

- #### En vez de hacer un `groupby().sum()`, después otro `groupby().mean()`, después otro `roupby().count()`, con agg hacés todo junto en una sola pasada.

### ¿Qué ganás?

- #### Menos código (más corto).

- #### Más ordenado (todo en la misma tabla).

- #### Más rápido (la compu hace el trabajo de una vez).

### El resultado es una tabla con todas las métricas lado a lado, lista para analizar o mostrar.
---

## 4) Ejemplo simple en código

### Ejemplo A — Préstamos por región

In [9]:
import pandas as pd

# Creamos un diccionario con datos de región, monto y cliente
data = {
    'Region': ['N', 'N', 'S', 'S', 'E', 'E', 'O', 'O'],
    'Monto': [1000, 2000, 1500, 3000, 2200, 1300, 2700, 1800],
    'Cliente': ['Ana', 'Luis', 'Sofia', 'Marta', 'Juan', 'Carla', 'Pedro', 'Laura']
}

df = pd.DataFrame(data)  # Convertimos el diccionario en un DataFrame (tabla)

print(df) # Mostramos la tabla original con todos los datos

print('')

# Agrupamos por región y calculamos suma, promedio y cantidad en una sola instrucción
resumen = df.groupby('Region').agg(
    Total_Monto=('Monto','sum'), # Suma de montos por región
    Promedio_Monto=('Monto','mean'),  # Promedio de montos por región
    Operaciones=('Cliente','count') # Cantidad de clientes en cada región
    ).reset_index() # Convierte Region otra vez en columna normal

print(resumen)

  Region  Monto Cliente
0      N   1000     Ana
1      N   2000    Luis
2      S   1500   Sofia
3      S   3000   Marta
4      E   2200    Juan
5      E   1300   Carla
6      O   2700   Pedro
7      O   1800   Laura

  Region  Total_Monto  Promedio_Monto  Operaciones
0      E         3500          1750.0            2
1      N         3000          1500.0            2
2      O         4500          2250.0            2
3      S         4500          2250.0            2


### Ejemplo B — Multi-columna (Región y Tipo)

In [12]:
# Creamos un diccionario con columnas: Región, Tipo de cliente y Monto del préstamo
data = {
    'Region': ['N', 'N', 'S', 'S', 'E', 'E', 'O', 'O'],
    'Tipo': ["Minorista","Corporativo","Minorista","Corporativo","Minorista","Corporativo","Minorista","Corporativo"],
    'Monto': [1000, 2000, 1500, 3000, 2200, 1300, 2700, 1800]
}

df = pd.DataFrame(data) # Convertimos el diccionario en un DataFrame (tabla de pandas)

# Agrupamos por Región y Tipo, luego aplicamos varias agregaciones con agg()
resumen = df.groupby(['Region', 'Tipo']).agg(
    Total_Monto=('Monto','sum'), # Suma de montos por cada grupo (Region+Tipo)
    Promedio_Monto=('Monto','mean'), # Promedio de montos por cada grupo
    Operaciones=('Monto','count') # Cantidad de registros en cada grupo
).reset_index() # Convertimos el índice de nuevo en columnas normales

print(resumen)

  Region         Tipo  Total_Monto  Promedio_Monto  Operaciones
0      E  Corporativo         1300          1300.0            1
1      E    Minorista         2200          2200.0            1
2      N  Corporativo         2000          2000.0            1
3      N    Minorista         1000          1000.0            1
4      O  Corporativo         1800          1800.0            1
5      O    Minorista         2700          2700.0            1
6      S  Corporativo         3000          3000.0            1
7      S    Minorista         1500          1500.0            1


### 5) Errores típicos

## **1. Olvidar usar diccionario en `agg`**

- #### Si escribís `agg(["sum","mean"])`, las columnas se llaman *sum* y *mean* → nombres genéricos y poco claros.
- #### Con diccionario podés dar nombres descriptivos:

  ```python
  df.groupby("Region").agg(
      Total_Monto=("Monto","sum"),
      Promedio_Monto=("Monto","mean")
  )
  ```

#### Esto hace que el resultado sea entendible y presentable. Además podés combinar distintas columnas y funciones en el mismo diccionario.

#### **2. Contar con `count()` en columnas con nulos**

#### `count()` solo cuenta valores **no nulos** → si hay `NaN`, el resultado es menor al número real de filas.
####  Si querés **todas las filas**, uses o no datos válidos, usá `size()`.
###  Diferencia:
- #### `count()` = “¿cuántos datos válidos tengo en esta columna?”
- #### `size()` = “¿cuántas filas hay en este grupo, con o sin datos faltantes?”

---


## 6) Ejercicios

#### Ejercicio 1 — Ventas por Sucursal y Categoría
#### Dataset: Sucursal, Categoria, Precio, Cantidad.

#### 1. Crear columna Ingreso = Precio * Cantidad.

#### 2. Agrupar por Sucursal y Categoria.

#### 3. Con agg, calcular:

- #### Total de Ingreso `(sum)`.

- #### Ingreso promedio `(mean)`.

- #### Cantidad de operaciones `(count)`.

#### 4. Interpretar: ¿qué combinación factura más y cuál tiene el ingreso promedio más alto?

In [15]:
import pandas as pd

# creamos el dataset 
data = {
    "Sucursal": ["Norte","Norte","Norte","Sur","Sur","Este","Este","Oeste","Oeste"],
    "Categoria": ["Electrónica","Ropa","Ropa","Electrónica","Ropa","Ropa","Electrónica","Ropa","Electrónica"],
    "Precio": [1000,500,800,1200,700,400,900,600,1500],
    "Cantidad": [2,3,1,1,5,2,3,4,1]
}

df = pd.DataFrame(data)

print(df)

print('')

# 2) creamos la columna Ingreso
df['Ingreso'] = df['Precio'] * df['Cantidad']

# 2) agrupamos por sucursal y cantidad
resumen = df.groupby(['Sucursal','Categoria']).agg(
    Total_Ingreso=('Ingreso','sum'),
    Promedio_Ingreso=('Ingreso','mean'),
    Operaciones=('Ingreso','count')
).reset_index()


print(resumen)

# 5) Interpretación

# La combinación que más factura (Total_Ingreso) → Sur + Ropa = 3500

# La que tiene mayor ingreso promedio (Promedio_Ingreso) → Sur + Ropa = 3500

  Sucursal    Categoria  Precio  Cantidad
0    Norte  Electrónica    1000         2
1    Norte         Ropa     500         3
2    Norte         Ropa     800         1
3      Sur  Electrónica    1200         1
4      Sur         Ropa     700         5
5     Este         Ropa     400         2
6     Este  Electrónica     900         3
7    Oeste         Ropa     600         4
8    Oeste  Electrónica    1500         1

  Sucursal    Categoria  Total_Ingreso  Promedio_Ingreso  Operaciones
0     Este  Electrónica           2700            2700.0            1
1     Este         Ropa            800             800.0            1
2    Norte  Electrónica           2000            2000.0            1
3    Norte         Ropa           2300            1150.0            2
4    Oeste  Electrónica           1500            1500.0            1
5    Oeste         Ropa           2400            2400.0            1
6      Sur  Electrónica           1200            1200.0            1
7      Sur         

# Caso práctico: Promedio de préstamo y % mora por región

## 1. Introducción

#### vamos a trabajar con un análisis muy típico en el mundo financiero: entender cómo se comportan los préstamos en distintas regiones.
#### Un banco o financiera no solo quiere saber cuánto dinero prestó, sino también:

- #### ¿Cuál es el monto promedio que la gente pide?

- #### ¿Qué porcentaje de esos préstamos entra en mora (es decir, clientes que no pagan a tiempo)?

#### Estos indicadores son clave porque ayudan a tomar decisiones: dónde conviene prestar más, dónde hay más riesgo, y cómo manejar las políticas de crédito en cada zona.
---

## 2. necesidad 

### Imaginemos que yo trabajo en el área de riesgo crediticio de un banco.
#### mi jefe me llama y me dice:

#### Necesito un informe urgente: quiero ver por región cuál es el promedio de préstamo y el porcentaje de mora. Así podremos decidir dónde expandir nuestras operaciones y dónde tener más cuidado.

#### Ahí surge la necesidad: tenemos que calcular dos métricas al mismo tiempo, agrupadas por región.
---

#### Para resolver este tipo de problemas en pandas, usamos dos pasos:

#### 1. `groupby()` → agrupar los datos por una columna (en este caso, “Region”).

#### 2. `.agg()` → calcular varias métricas a la vez (promedios, sumas, porcentajes, etc.).

### 📌 Concepto importante:

- #### Cuando la columna tiene números continuos (ejemplo: monto), podemos calcular mean, sum, max, etc.

- #### Cuando la columna tiene 0 y 1 (ejemplo: mora: 0 = paga, 1 = en mora), el mean devuelve el porcentaje de morosos, porque calcula la proporción.
---

## Ejemplo 1

In [16]:
import pandas as pd

# Dataset sencillo
data = {
    "Region": ["Norte","Norte","Sur","Sur","Este","Este"],
    "Monto":  [2000, 2500, 1200, 1000, 3000, 2800],
    "Mora":   [0, 1, 0, 1, 0, 0]  # 0 = paga, 1 = en mora
}
df = pd.DataFrame(data)

resumen = df.groupby("Region").agg(
    Promedio_Prestamo=("Monto","mean"),
    Porcentaje_Mora=("Mora","mean")
).reset_index()

# Convertimos a porcentaje
resumen["Porcentaje_Mora"] = resumen["Porcentaje_Mora"] * 100

print(resumen)


  Region  Promedio_Prestamo  Porcentaje_Mora
0   Este             2900.0              0.0
1  Norte             2250.0             50.0
2    Sur             1100.0             50.0


## Ejemplo 2 (con más columnas)

#### Ahora agregamos los clientes para ver que se pueden incluir más datos.

In [17]:
data = {
    "Region":   ["Norte","Norte","Sur","Sur","Este","Este","Oeste","Oeste"],
    "Cliente":  ["A","B","C","D","E","F","G","H"],
    "Monto":    [2000,1500,1200,1000,3000,2800,1700,1600],
    "Mora":     [0,1,1,0,0,0,1,0]
}
df = pd.DataFrame(data)

resumen = df.groupby("Region").agg(
    Promedio_Prestamo=("Monto","mean"), # promedio de préstamo
    Total_Prestamos=("Monto","sum"),    # suma total prestada
    Porcentaje_Mora=("Mora","mean")     # porcentaje de mora (promedio de 0 y 1)
).reset_index()

# Convertimos la mora en porcentaje (ej: 0.25 → 25%)
resumen["Porcentaje_Mora"] = resumen["Porcentaje_Mora"] * 100
print(resumen)

# Ahora tenemos: promedio de préstamo, total prestado y % de mora.
# Esto se parece mucho más a un informe real de un banco.


  Region  Promedio_Prestamo  Total_Prestamos  Porcentaje_Mora
0   Este             2900.0             5800              0.0
1  Norte             1750.0             3500             50.0
2  Oeste             1650.0             3300             50.0
3    Sur             1100.0             2200             50.0


## Ejemplo 3 (ordenando resultados)
#### Podemos ordenar las regiones según el riesgo (mayor % de mora primero).

In [18]:
# Ordenamos las regiones por % de mora (de mayor a menor)
resumen_ordenado = resumen.sort_values(by="Porcentaje_Mora", ascending=False) 
print(resumen_ordenado)


  Region  Promedio_Prestamo  Total_Prestamos  Porcentaje_Mora
1  Norte             1750.0             3500             50.0
2  Oeste             1650.0             3300             50.0
3    Sur             1100.0             2200             50.0
0   Este             2900.0             5800              0.0


## Ejercicios propuestos

#### 1. **Ventas por región**
   Dataset: Región, Producto, Precio, Cantidad, Deuda (0 = pagó, 1 = debe).

- #### Crear columna Ingreso = Precio \* Cantidad.
- #### Agrupar por Región.
- #### Calcular:

- ####   Promedio de Ingreso.
- ####   Total de Ingreso.
- ####   % de clientes con deuda.

#### 2. **Créditos por sucursal**
   Dataset: Sucursal, Cliente, Monto, Mora.

- #### Agrupar por sucursal.
- #### Calcular el monto promedio, el monto máximo y el % de mora.
- #### Ordenar las sucursales por monto promedio descendente.

#### 3. **Análisis extra**
####    Con el dataset de ejemplo, calcular además **cuántos clientes tiene cada región** y mostrarlo junto al promedio y % de mora.

---
---
---

## Ejercicios

### 1. Ventas por región
#### Dataset: Región, Producto, Precio, Cantidad, Deuda (0 = pagó, 1 = debe).

- #### Crear columna Ingreso = Precio * Cantidad.

- #### Agrupar por Región.

- #### Calcular:

#### Promedio de Ingreso.

#### Total de Ingreso.

#### % de clientes con deuda.

In [19]:
import pandas as pd

# Dataset: Región, Producto, Precio, Cantidad, Deuda (0=pagó, 1=debe)
data = {
    "Region":   ["Norte","Norte","Sur","Sur","Este","Este","Oeste","Oeste"],
    "Producto": ["A","B","A","C","B","C","A","B"],
    "Precio":   [100, 150,  80, 120, 200, 180, 90, 160],
    "Cantidad": [  5,   2,  10,   1,   3,   2,  4,   1],
    "Deuda":    [  0,   1,   0,   1,   0,   0,  1,   0]
}
df = pd.DataFrame(data)

# 1) Columna Ingreso = Precio * Cantidad
df["Ingreso"] = df["Precio"] * df["Cantidad"]

# 2) Agrupar por Región y calcular métricas
resumen = df.groupby("Region").agg(
    Promedio_Ingreso=("Ingreso","mean"),     # promedio por operación
    Total_Ingreso=("Ingreso","sum"),         # suma total
    Porcentaje_Deuda=("Deuda","mean")        # proporción de operaciones con deuda
).reset_index()

# 3) Convertir proporción a porcentaje
resumen["Porcentaje_Deuda"] = resumen["Porcentaje_Deuda"] * 100

print(resumen)
# Nota: el % de deuda aquí es por operación (no por cliente, porque no hay columna Cliente).


  Region  Promedio_Ingreso  Total_Ingreso  Porcentaje_Deuda
0   Este             480.0            960               0.0
1  Norte             400.0            800              50.0
2  Oeste             260.0            520              50.0
3    Sur             460.0            920              50.0


## 2. Créditos por sucursal
### Dataset: Sucursal, Cliente, Monto, Mora.

- #### Agrupar por sucursal.

- #### Calcular el monto promedio, el monto máximo y el % de mora.

- #### Ordenar las sucursales por monto promedio descendente.

In [20]:
import pandas as pd

# Dataset: Sucursal, Cliente, Monto, Mora (0=paga, 1=en mora)
data = {
    "Sucursal": ["Centro","Centro","Centro","Norte","Norte","Sur","Sur","Sur"],
    "Cliente":  ["A","B","C","D","E","F","G","H"],
    "Monto":    [2000, 1500, 3000, 1800, 2200, 1000, 1200, 900],
    "Mora":     [0,    1,    0,    0,    1,    0,    1,    0]
}
df = pd.DataFrame(data)

# 1) Agrupar por Sucursal y calcular métricas
resumen = df.groupby("Sucursal").agg(
    Monto_Promedio=("Monto","mean"),   # promedio
    Monto_Maximo=("Monto","max"),      # máximo
    Porcentaje_Mora=("Mora","mean")    # proporción de clientes en mora
).reset_index()

# 2) Convertir proporción a %
resumen["Porcentaje_Mora"] = resumen["Porcentaje_Mora"] * 100

# 3) Ordenar por monto promedio (descendente)
resumen_ordenado = resumen.sort_values(by="Monto_Promedio", ascending=False)

print(resumen_ordenado)


  Sucursal  Monto_Promedio  Monto_Maximo  Porcentaje_Mora
0   Centro     2166.666667          3000        33.333333
1    Norte     2000.000000          2200        50.000000
2      Sur     1033.333333          1200        33.333333


## 3. Análisis extra
#### Con el dataset de ejemplo, calcular además cuántos clientes tiene cada región y mostrarlo junto al promedio y % de mora.

In [ ]:
import pandas as pd

# Dataset de ejemplo: Region, Cliente, Monto, Mora
data = {
    "Region":  ["Norte","Norte","Norte","Sur","Sur","Este","Este","Oeste","Oeste"],
    "Cliente": ["A","B","A","C","D","E","F","G","H"],  # un cliente puede aparecer más de una vez
    "Monto":   [2000,1500,1800,1200,1000,3000,2800,1700,1600],
    "Mora":    [0,1,0,1,0,0,0,1,0]
}
df = pd.DataFrame(data)

# 1) Métricas principales por región
resumen = df.groupby("Region").agg(
    Promedio_Prestamo=("Monto","mean"),
    Porcentaje_Mora=("Mora","mean")
).reset_index()

# 2) Contar clientes únicos por región (no repite el mismo cliente)
clientes_region = df.groupby("Region")["Cliente"].nunique().reset_index(name="Clientes_Unicos")

# 3) Unir métricas + cantidad de clientes
resumen = resumen.merge(clientes_region, on="Region", how="left")

# 4) Convertir mora a %
resumen["Porcentaje_Mora"] = resumen["Porcentaje_Mora"] * 100

print(resumen)
# Ahora ves: promedio, % mora y cuántos clientes únicos tiene cada región.
